# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

# TensorFLow includes a data provider for MNIST that we'll use.
# It comes with the tensorflow-datasets module, therefore, if you haven't please install the package using
# pip install tensorflow-datasets 
# or
# conda install tensorflow-datasets

import tensorflow_datasets as tfds

# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

## Data & Preprocessing

That's where we load and preprocess our data.

In [57]:
# remember the comment from above
# these datasets will be stored in C:\Users\*USERNAME*\tensorflow_datasets\...
# the first time you download a dataset, it is stored in the respective folder 
# every other time, it is automatically loading the copy on your computer 

# tfds.load actually loads a dataset (or downloads and then loads if that's the first time you use it) 
# in our case, we are interesteed in the MNIST; the name of the dataset is the only mandatory argument
# there are other arguments we can specify, which we can find useful
# mnist_dataset = tfds.load(name='mnist', as_supervised=True)
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# with_info=True will also provide us with a tuple containing information about the version, features, number of samples
# we will use this information a bit below and we will store it in mnist_info

# as_supervised=True will load the dataset in a 2-tuple structure (input, target) 
# alternatively, as_supervised=False, would return a dictionary
# obviously we prefer to have our inputs and targets separated 

# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

# by default, TF has training and testing datasets, but no validation sets
# thus we must split it on our own

# we start by defining the number of validation samples as a % of the train samples
# this is also where we make use of mnist_info (we don't have to count the observations)
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
# let's cast this number to an integer, as a float may cause an error along the way
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# let's also store the number of test samples in a dedicated variable (instead of using the mnist_info one)
num_test_samples = mnist_info.splits['test'].num_examples
# once more, we'd prefer an integer (rather than the default float)
num_test_samples = tf.cast(num_test_samples, tf.int64)


# normally, we would like to scale our data in some way to make the result more numerically stable
# in this case we will simply prefer to have inputs between 0 and 1
# let's define a function called: scale, that will take an MNIST image and its label
def scale(image, label):
    # we make sure the value is a float
    image = tf.cast(image, tf.float32)
    # since the possible values for the inputs are 0 to 255 (256 different shades of grey)
    # if we divide each element by 255, we would get the desired result -> all elements will be between 0 and 1 
    image /= 255.

    return image, label


# the method .map() allows us to apply a custom transformation to a given dataset
# we have already decided that we will get the validation data from mnist_train, so 
scaled_train_and_validation_data = mnist_train.map(scale)

# finally, we scale and batch the test data
# we scale it so it has the same magnitude as the train and validation
# there is no need to shuffle it, because we won't be training on the test data
# there would be a single batch, equal to the size of the test data
test_data = mnist_test.map(scale)


# let's also shuffle the data

BUFFER_SIZE = 10000
# this BUFFER_SIZE parameter is here for cases when we're dealing with enormous datasets
# then we can't shuffle the whole dataset in one go because we can't fit it all in memory
# so instead TF only stores BUFFER_SIZE samples in memory at a time and shuffles them
# if BUFFER_SIZE=1 => no shuffling will actually happen
# if BUFFER_SIZE >= num samples => shuffling is uniform
# BUFFER_SIZE in between - a computational optimization to approximate uniform shuffling

# luckily for us, there is a shuffle method readily available and we just need to specify the buffer size
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# once we have scaled and shuffled the data, we can proceed to actually extracting the train and validation
# our validation data would be equal to 10% of the training set, which we've already calculated
# we use the .take() method to take that many samples
# finally, we create a batch with a batch size equal to the total number of validation samples
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

# similarly, the train_data is everything else, so we skip as many samples as there are in the validation dataset
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

# determine the batch size
BATCH_SIZE = 100

# we can also take advantage of the occasion to batch the train data
# this would be very helpful when we train, as we would be able to iterate over the different batches
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

# batch the test data
test_data = test_data.batch(num_test_samples)


# takes next batch (it is the only batch)
# because as_supervized=True, we've got a 2-tuple structure
validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model
When thinking about a deep learning algorithm, we mostly imagine building the model. So, let's do it :)

In [64]:
input_size = 784
output_size = 10
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 200
    
# define how the model will look like
model = tf.keras.Sequential([
    
    # the first layer (the input layer)
    # each observation is 28x28x1 pixels, therefore it is a tensor of rank 3
    # since we don't know CNNs yet, we don't know how to feed such input into our net, so we must flatten the images
    # there is a convenient method 'Flatten' that simply takes our 28x28x1 tensor and orders it into a (None,) 
    # or (28x28x1,) = (784,) vector
    # this allows us to actually create a feed forward neural network
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)), # input layer
    
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    
    # the final layer is no different, we just make sure to activate it with softmax
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choose the optimizer and the loss function

In [65]:
# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
That's where we train the model we have built.

In [66]:
# determine the maximum number of epochs
NUM_EPOCHS = 5

# we fit the model, specifying the
# training data
# the total number of epochs
# and the validation data we just created ourselves in the format: (inputs,targets)
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose =2)

Epoch 1/5
54000/54000 - 135s - loss: 0.2546 - accuracy: 0.9306 - val_loss: 0.1447 - val_accuracy: 0.9552
Epoch 2/5
54000/54000 - 133s - loss: 0.1723 - accuracy: 0.9583 - val_loss: 0.1361 - val_accuracy: 0.9657
Epoch 3/5
54000/54000 - 132s - loss: 0.1639 - accuracy: 0.9632 - val_loss: 0.1486 - val_accuracy: 0.9698
Epoch 4/5
54000/54000 - 129s - loss: 0.1477 - accuracy: 0.9671 - val_loss: 0.1519 - val_accuracy: 0.9677
Epoch 5/5
54000/54000 - 126s - loss: 0.1404 - accuracy: 0.9687 - val_loss: 0.1538 - val_accuracy: 0.9678


# Exercises

There are several main adjustments you may try.

Please pay attention to the time it takes for each epoch to conclude.

Using the code from the lecture as the basis, fiddle with the hyperparameters of the algorithm.


**1. The *width* (the hidden layer size) of the algorithm. Try a hidden layer size of 200. How does the validation accuracy of the model change? What about the time it took the algorithm to train? Can you find a hidden layer size that does better?**

ANS: With a hidden layer size of 50 we obtained `val_accuracy: 0.9675`. Increasing the width of the algorith to a hidden layer size of 200 dramatically increases the accuracy of the model as we get to a `val_accuracy: 0.9882`.

**Solution**

Find the variable: "hidden_layer_size" and change it to 200.

The validation accuracy is significantly higher (as the algorithm with 50 hidden units was too simple of a model).

Naturally, it takes the algorithm much longer to train (unless early stopping is triggered too soon).

A hidden layer size of 500 (and not only) works even better.



**2. The *depth* of the algorithm. Add another hidden layer to the algorithm. This is an extremely important exercise! How does the validation accuracy change? What about the time it took the algorithm to train? Hint: Be careful with the shapes of the weights and the biases.**

ANS: Keeping the width unchanged (50), by adding an extra hidden layer to the model we have increased the accuracy to `val_accuracy: 0.973` on the fifth epoch. The training was slower, hoverver, it gradually got faster after each epoch.

**Solution**

Adding another hidden layer to the algorithm is done in the same way as in the lecture.

We simply add a new line in Sequential:

tf.keras.layers.Dense(hidden_layer_size, activation='relu')

We can see that the accuracy of the model does not necessarily improve. This is an important lesson for us. Fiddling with a single hyperparameter may not be enough. Sometimes, a deeper net needs to also be wider in order to have higher accuracy. Maybe you need more epochs?

**ADDITIONAL TASK: Try this new model, but with a wider one (200-500 hidden units). Basically, combine this and the previous exercises**

In any case, it takes longer for the algorithm to train.



**3. The *width and depth* of the algorithm. Add as many additional layers as you need to reach 5 hidden layers. Moreover, adjust the width of the algorithm as you find suitable. How does the validation accuracy change? What about the time it took the algorithm to train?**

ANS: Adding two more hidden layers to the model while keeping the width to 50 has made our model diverge after the 4th epoch. That's not good.

```
Epoch 1/5
540/540 - 4s - loss: 0.4204 - accuracy: 0.8661 - val_loss: 0.1773 - val_accuracy: 0.9475
Epoch 2/5
540/540 - 4s - loss: 0.1597 - accuracy: 0.9511 - val_loss: 0.1271 - val_accuracy: 0.9623
Epoch 3/5
540/540 - 4s - loss: 0.1196 - accuracy: 0.9632 - val_loss: 0.1102 - val_accuracy: 0.9673
Epoch 4/5
540/540 - 4s - loss: 0.1011 - accuracy: 0.9693 - val_loss: 0.0892 - val_accuracy: 0.9747
Epoch 5/5
540/540 - 4s - loss: 0.0848 - accuracy: 0.9741 - val_loss: 0.0892 - val_accuracy: 0.9742
```

Thus, we increase the number of nodes in the hidden layers to 100 (appeared to be an optimal width on Exercise 1) and we've solved that problem, however, the overall accuracy of the model is still lower than we had by simply increasing the width to 200 and keeping depth to 2.

Finally, increasing the width to 200 we get a `val_accuracy: 0.9852` on the 5th epoch but the training is slightly slower. by removing a hidden layer we make our model more accurate and reduce the training speed (`val_accuracy: 0.9870` width: 200, depth: 3).

**Solution**

This exercise is pretty much the same as the previous one. However, it will get us to a much deeper net. As we noted in the previous exercise, you a deeeper net may need to be wider to produce better results.

We tried with 1000 hidden units in each layer and 5 hidden layers.

The result (as you can see below) is that our model's training was going very well, until it overfit. It did so by quite a lot.

It took my personal computer around 5-6 minutes to train the model. 

What if you have more epochs?



**4. Fiddle with the activation functions. Try applying sigmoid transformation to both layers. The sigmoid activation is given by the string 'sigmoid'.**

ANS: Using the sigmoid activation function instead of ReLu has decreased the accuracy of the model drastically.

```
Epoch 1/5
540/540 - 4s - loss: 1.0050 - accuracy: 0.7818 - val_loss: 0.4209 - val_accuracy: 0.9010
Epoch 2/5
540/540 - 4s - loss: 0.3309 - accuracy: 0.9134 - val_loss: 0.2638 - val_accuracy: 0.9287
Epoch 3/5
540/540 - 4s - loss: 0.2397 - accuracy: 0.9322 - val_loss: 0.2119 - val_accuracy: 0.9395
Epoch 4/5
540/540 - 4s - loss: 0.1949 - accuracy: 0.9442 - val_loss: 0.1781 - val_accuracy: 0.9490
Epoch 5/5
540/540 - 4s - loss: 0.1646 - accuracy: 0.9531 - val_loss: 0.1523 - val_accuracy: 0.9580
```

**Solution**

Find the part where we stack layers (Sequential()).

Adjust the activations from 'relu' to 'sigmoid'.
    
Generally, we should reach an inferior solution. That is because relu 'cleans' the noise in the data (think about it - if a value is negative, relu filters it out, while if it is positive, it takes it into account). For the MNIST dataset, we care only about the intensely black and white parts in the images of the digits, so such filtering proves beneficial.

The sigmoid does not filter the signals as well as relu, but still reaches a respectable result (around 95%).



**5. Fiddle with the activation functions. Try applying a ReLu to the first hidden layer and tanh to the second one. The tanh activation is given by the string 'tanh'.**

ANS: Keeping the original size (width: 50, depth: 2) the model yileds a higher accuracy.

```
Epoch 1/5
540/540 - 5s - loss: 0.4111 - accuracy: 0.8871 - val_loss: 0.1912 - val_accuracy: 0.9453
Epoch 2/5
540/540 - 4s - loss: 0.1676 - accuracy: 0.9502 - val_loss: 0.1314 - val_accuracy: 0.9613
Epoch 3/5
540/540 - 4s - loss: 0.1254 - accuracy: 0.9619 - val_loss: 0.1124 - val_accuracy: 0.9673
Epoch 4/5
540/540 - 3s - loss: 0.1001 - accuracy: 0.9705 - val_loss: 0.0942 - val_accuracy: 0.9718
Epoch 5/5
540/540 - 4s - loss: 0.0846 - accuracy: 0.9748 - val_loss: 0.0834 - val_accuracy: 0.9758
```

**Solution** 

Analogically to the previous lecture, we can change the activation functions. This time though, we will use different activators for the different layers.

The result should not be significantly different. However, with different width and depth, that may change.

*Additional exercise: Try to find a better combination of activation functions*



**6. Adjust the batch size. Try a batch size of 10000. How does the required time change? What about the accuracy?**

The training is reduced to 2-3 seconds per epoch, however the overall accuracy of the model is much worse as we obtained a `val_accuracy: 0.8123` after the 5th epoch.

**Solution**

Find the line that declares the batch size.

Change batch_size from 100 to 10000.

    BATCH_SIZE = 10000
    
A bigger batch size results in slower training. That's what we expected from the theory. We are taking advantage of batching because of the amazing speed increase.

Notice that the validation accuracy starts from a low number and with 5 epochs actually **finishes** at a lower number. That's because there are **fewer** updates in a single epoch.

*Try a batch size of 30,000 or 50,000. That's very close to single batch GD for this problem. What do you think about the speed?You will need to change the max epochs to 100 (for instance), as 5 epochs won't be enough to train the model. What do you think about the speed of optimization?*



**7. Adjust the batch size. Try a batch size of 1. That's the SGD. How do the time and accuracy change? Is the result coherent with the theory?**

The result is coherent with the theory we've seen before. Recall:

- batch size = 1 is the SGD (very accurate but very slow as essentially we are creating as many batches as raws of data. We're running the model for every single observation in the sample.
- batch size = # samples is the same as running one single sample. The result we get is not very accurate.
- 1 < batch size < # samples (mini-batch Gradient Descent)

With a batch of size 1 we got a far better accuracy result, but the time it took to train the model was way too slow.

```
Epoch 1/5
54000/54000 - 115s - loss: 0.2512 - accuracy: 0.9252 - val_loss: 0.1516 - val_accuracy: 0.9563
Epoch 2/5
54000/54000 - 105s - loss: 0.1613 - accuracy: 0.9552 - val_loss: 0.1312 - val_accuracy: 0.9630
Epoch 3/5
54000/54000 - 93s - loss: 0.1447 - accuracy: 0.9618 - val_loss: 0.1641 - val_accuracy: 0.9598
Epoch 4/5
54000/54000 - 90s - loss: 0.1358 - accuracy: 0.9643 - val_loss: 0.1192 - val_accuracy: 0.9710
Epoch 5/5
54000/54000 - 89s - loss: 0.1342 - accuracy: 0.9667 - val_loss: 0.1376 - val_accuracy: 0.9717
```

**Solution**

Find the line that declares the batch size.

Change batch_size from 100 to 1.

    batch_size = 1
    
A batch size of 1 results in the SGD. It takes the algorithm very little time to process a single batch (as it is one data point), but there are thousands of batches (54000 to be precise), thus the algorithm is actually slow. Remember that this depends on the number of cores that you train on. If you are using a CPU with 4 or 8 cores, you can only train 4 or 8 batches at once. The middle ground (mini-batching such as 100 samples per batch) is optimal.

Notice that the validation accuracy starts from a high number. That's because there are **lots** updates in a single epoch. Once the training is over, the accuracy is lower than all other batch sizes (SGD was an approximation).



**8. Adjust the learning rate. Try a value of 0.0001. Does it make a difference?**

**Solution**

First, we have to define a custom optimizer (as we did in the TensorFlow intro).

We create the custom optimizer with:

    custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

Then we change the respective argument in model.compile to reflect this: 

    model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    

Since the learning rate is lower than normal, we may need to adjust the max_epochs (to, say, 50). 

The result is basically the same, but we reach it much slower.

While Adam adapts to the problem, if the orders of magnitude are too different, it may not have enough time to adjust accordingly.



**9. Adjust the learning rate. Try a value of 0.02. Does it make a difference?**

**Solution**

First, we have to define a custom optimizer (as we did in the TensorFlow intro).

We create the custom optimizer with:

    custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

Then we change the respective argument in model.compile to reflect this: 

    model.compile(optimizer=custom_optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    

While Adam adapts to the problem, if the orders of magnitude are too different, it may not have time to adjust accordingly. We start overfitting before we can reach a neat solution.

Therefore, for this problem, even 0.02 is a **HIGH** starting learning rate. What if you try a learning rate of = 1?

It's a good practice to try 0.001, 0.0001, and 0.00001. If it makes no difference, pick whatever, otherwise it makes sense to fiddle with the learning rate.



**10. Combine all the methods above and try to reach a validation accuracy of 98.5+ percent.**

Achieving 98.5% accuracy with the methodology we've seen so far is extremely hard. A more realistic exercise would be to achieve 98%+ accuracy. However, being pushed to the limit (trying to achieve 98.5%), you have probably learned a whole lot about the machine learning process.

Here is a link where you can check the results that some leading academics got on the MNIST (using different methodologies):
https://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results

**Solution**

After some fine tuning, I decided to brute-force the algorithm and created 10 hidden layers with 5000 hidden units each. 

    hidden_layer_size = 5000
    batch_size = 150
    NUM_EPOCHS = 10
    
All activation functions are ReLu.

There are better solutions using this methodology, this one is just superior to the one in the lessons. Due to the width and the depth of the algorithm, it took my computer 3 hours and 50 mins to train it. Good luck!

## Test the model

As we discussed in the lectures, after training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. You should not test before you are completely done with adjusting your model.

If you adjust your model after testing, you will start overfitting the test dataset, which will defeat its purpose.

In [9]:
test_loss, test_accuracy = model.evaluate(test_data)

      1/Unknown - 1s 1s/step - loss: 0.0897 - accuracy: 0.97 - 1s 1s/step - loss: 0.0897 - accuracy: 0.9728

In [10]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.28%


Using the initial model and hyperparameters given in this notebook, the final test accuracy should be roughly around 97%.

Each time the code is rerun, we get a different accuracy as the batches are shuffled, the weights are initialized in a different way, etc.

Finally, we have intentionally reached a suboptimal solution, so you can have space to build on it.